In [2]:
import mne
import os.path as op
from scipy.stats import *
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
path = "/cubric/scratch/c1557187/act_mis/"
subj = "0009"

In [4]:
epochs = mne.read_epochs(op.join(path, "MEG", subj, "all-000-epo.fif"))
inv = mne.minimum_norm.read_inverse_operator(op.join(path, "MEG", subj, "inv-000-inv.fif"))

In [ ]:
epochs

In [5]:
trig_dict = {
    "standard": ["30"],
    "odd": ["40"]
}

# movement = mne.EvokedArray(trim_mean(epochs, 0.1, axis=0), epochs.info).filter(1,40)
# movement.crop(-0.5, 1.5)

# all_stuff = mne.EvokedArray(trim_mean(epochs, 0.1, axis=0), epochs.info, tmin=-0.5).filter(1,40)

observation = {}

for k in trig_dict.keys():
    i = trig_dict[k]
    evo = mne.EvokedArray(trim_mean(epochs[i].get_data(), 0.1, axis=0),epochs.info).filter(1,40)
    evo.crop(1.4, 2.5)
    evo.shift_time(-1.5)    
    observation[k] = evo

In [6]:
# sensor = "MZC02-3305"
# mne.viz.plot_compare_evokeds(observation, observation["standard"].ch_names.index(sensor));
diff = mne.combine_evoked([observation["standard"], - observation["odd"]], weights="equal")

In [7]:
method_dict = {
    "dSPM": (8, 12, 15),
    "sLORETA": (3, 5, 7),
    "eLORETA": (0.75, 1.25, 1.75)
}

method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
lims = method_dict[method]

stc = mne.minimum_norm.apply_inverse(
    diff,
    inv,
    lambda2,
    method=method,
    pick_ori=None,
    verbose=True
)

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 2
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 262 (12 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "0.500 * unknown + 0.500 * -unknown"...
    Picked 274 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  37.5% variance
    Combining the current components...
    dSPM...
[done]


In [ ]:
stc.plot(subjects_dir=op.join(path, "MRI"), subject="0009", time_viewer=True, hemi="both");